<img src="http://wandb.me/logo-im-png" width="400" alt="Weights & Biases" />
<!--- @wandbcode{course-cicd} -->

<a href="https://colab.research.google.com/github/hamelsmu/wandb-cicd/blob/main/intro/create_runs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CI/CD For Machine Learning Course

[Course website](https://www.wandb.courses/courses/ci-cd-for-machine-learning/)

This notebook creates synthetic runs so you can explore the W&B user interface.

After you create runs, please login to the W&B user interface and explore it a bit.  This will give you important context for the course.

## Setup

First, let's install `wandb`:

In [4]:
!pip install wandb
import wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 16.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.0 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=c37bf7feb312cdb5dc640c79046d4fd07535715972c0b6744f3f908fb5714575
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [5]:
#@title ## Log Runs { run: "auto", display-mode: "form" }
#@markdown This will create a Weights & Biases project for you and log runs to it.
#@markdown Please choose a project name, or just leave the default one as is.  This can take 3-4 minutes to run.

LOG_DUMMY_RUNS = True


import requests
from PIL import Image
from io import BytesIO
import wandb
import pandas as pd
from itertools import product
import random
import math

import wandb
import random
import string

ENTITY = wandb.apis.PublicApi().default_entity
PROJECT = "cicd-quickstart\"" #@param {type: "string"}
LINEAGE_PROJECT = "lineage-example"


def get_image(url):
    r = requests.get(url)
    return Image.open(BytesIO(r.content))


def log_dummy_data():
    run_names = [
        "adventurous-aardvark-1",
        "bountiful-badger-2",
        "clairvoyant-chipmunk-3",
        "dastardly-duck-4",
        "eloquent-elephant-5",
        "flippant-flamingo-6",
        "giddy-giraffe-7",
        "haughty-hippo-8",
        "ignorant-iguana-9",
        "jolly-jackal-10",
        "kind-koala-11",
        "laughing-lemur-12",
        "manic-mandrill-13",
        "neighbourly-narwhal-14",
        "oblivious-octopus-15",
        "philistine-platypus-16",
        "quant-quail-17",
        "rowdy-rhino-18",
        "solid-snake-19",
        "timid-tarantula-20",
        "understanding-unicorn-21",
        "voracious-vulture-22",
        "wu-tang-23",
        "xenic-xerneas-24",
        "yielding-yveltal-25",
        "zooming-zygarde-26",
    ]

    opts = ["adam", "sgd"]
    encoders = ["resnet18", "resnet50"]
    learning_rates = [0.01]
    for (i, run_name), (opt, encoder, lr) in zip(
        enumerate(run_names), product(opts, encoders, learning_rates)
    ):
        config = {
            "optimizer": opt,
            "encoder": encoder,
            "learning_rate": lr,
            "momentum": 0.1 * random.random(),
        }
        displacement1 = random.random() * 2
        displacement2 = random.random() * 4
        with wandb.init(
            entity=ENTITY, project=PROJECT, config=config, name=run_name
        ) as run:
            for step in range(1000):
                wandb.log(
                    {
                        "acc": 0.1
                        + 0.4
                        * (
                            math.log(1 + step + random.random())
                            + random.random() * run.config.learning_rate
                            + random.random()
                            + displacement1
                            + random.random() * run.config.momentum
                        ),
                        "val_acc": 0.1
                        + 0.4
                        * (
                            math.log(1 + step + random.random())
                            + random.random() * run.config.learning_rate
                            - random.random()
                            + displacement1
                        ),
                        "loss": 0.1
                        + 0.08
                        * (
                            3.5
                            - math.log(1 + step + random.random())
                            + random.random() * run.config.momentum
                            + random.random()
                            + displacement2
                        ),
                        "val_loss": 0.1
                        + 0.04
                        * (
                            4.5
                            - math.log(1 + step + random.random())
                            + random.random() * run.config.learning_rate
                            - random.random()
                            + displacement2
                        ),
                    }
                )

    with wandb.init(
        entity=ENTITY, project=PROJECT, config=config, name=run_names[i + 1]
    ) as run:
        img = get_image(
            "https://www.akc.org/wp-content/uploads/2017/11/Shiba-Inu-standing-in-profile-outdoors.jpg"
        )
        image = wandb.Image(img)
        df = pd.DataFrame(
            {
                "int": [1, 2, 3, 4],
                "float": [1.2, 2.3, 3.4, 4.5],
                "str": ["a", "b", "c", "d"],
                "img": [image] * 4,
            }
        )
        run.log({"img": image, "my-table": df})


class Step:
    def __init__(self, j, r, u, o, at=None):
        self.job_type = j
        self.runs = r
        self.uses_per_run = u
        self.outputs_per_run = o
        self.artifact_type = at if at is not None else "model"
        self.artifacts = []


def create_artifact(name: str, type: str, content: str):
    art = wandb.Artifact(name, type)
    with open("boom.txt", "w") as f:
        f.write(content)
    art.add_file("boom.txt", "test-name")

    img = get_image(
        "https://www.akc.org/wp-content/uploads/2017/11/Shiba-Inu-standing-in-profile-outdoors.jpg"
    )
    image = wandb.Image(img)
    df = pd.DataFrame(
        {
            "int": [1, 2, 3, 4],
            "float": [1.2, 2.3, 3.4, 4.5],
            "str": ["a", "b", "c", "d"],
            "img": [image] * 4,
        }
    )
    art.add(wandb.Table(dataframe=df), "dataframe")
    return art


def log_dummy_lineage():
    pipeline = [
        Step("dataset-generator", 1, 0, 3, "dataset"),
        Step("trainer", 4, (1, 2), 3),
        Step("evaluator", 2, 1, 3),
        Step("ensemble", 1, 1, 1),
    ]
    for (i, step) in enumerate(pipeline):
        for _ in range(step.runs):
            with wandb.init(project=LINEAGE_PROJECT, job_type=step.job_type) as run:
                # use
                uses = step.uses_per_run
                if type(uses) == tuple:
                    uses = random.choice(list(uses))

                if i > 0:
                    prev_step = pipeline[i - 1]
                    input_artifacts = random.sample(prev_step.artifacts, uses)
                    for a in input_artifacts:
                        run.use_artifact(a)
                # log output artifacts
                for j in range(step.outputs_per_run):
                    # name = ''.join(random.choices(string.ascii_lowercase + string.digits, k=6))
                    name = f"{step.artifact_type}-{j}"
                    content = "".join(
                        random.choices(string.ascii_lowercase + string.digits, k=12)
                    )
                    art = create_artifact(name, step.artifact_type, content)
                    run.log_artifact(art)
                    art.wait()

                    # save in pipeline
                    step.artifacts.append(art)

if LOG_DUMMY_RUNS:
  log_dummy_data()
  log_dummy_lineage()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: suka (piedpipers). Use `wandb login --relogin` to force relogin


acc,▁▂▄▄▅▅▅▅▅▆▆▆▆▆▅▆▆▆▇▆▆▇▇▆▇▆▇▇▇▇▇▇▇█▇█▇█▇█
loss,█▆▅▄▄▃▃▃▃▃▄▃▃▃▃▃▃▂▂▂▃▂▂▂▂▂▂▂▂▂▁▁▂▂▂▂▂▁▁▁
val_acc,▁▃▄▄▅▅▆▆▆▆▇▆▆▆▇▆▇▆▇▇▆▇█▇▇▇▇██▇███▇▇▇█▇██
val_loss,█▆▅▅▅▄▅▄▄▃▃▃▃▃▂▂▃▂▂▃▂▁▂▃▁▃▂▃▂▂▂▂▁▂▂▂▁▁▂▁
acc,3.73202
loss,0.14299
val_acc,3.42903
val_loss,0.11459


acc,▁▂▄▄▅▄▅▅▆▅▅▆▆▆▆▇▆▇▆▇▇▆▆▇▆▇▇▆▇▇▇█▇▇███▇█▇
loss,█▆▅▄▄▄▃▄▄▃▄▄▃▄▃▃▃▂▃▃▃▂▂▃▁▂▂▂▂▂▂▂▂▂▁▁▂▂▂▁
val_acc,▁▃▃▄▅▅▅▅▅▅▆▆▆▆▆▇▇▆▇▆▆▇▇▇▇▇▇▇▇▇▇██▇█████▇
val_loss,█▇▆▅▅▄▄▄▄▃▄▃▃▃▃▃▂▂▂▃▂▂▃▂▂▂▂▁▂▂▂▂▂▂▁▂▁▁▁▁
acc,3.76707
loss,0.21778
val_acc,3.24734
val_loss,0.15471


acc,▁▃▄▃▄▅▅▅▅▅▅▆▆▅▇▆▆▆▆▆▇▇▇▇▇▇█▇▇▇▇▇▆█▇▇██▇█
loss,█▅▆▅▅▄▃▃▃▃▂▃▂▃▂▂▂▂▂▃▂▂▁▁▂▂▁▂▂▁▂▂▂▂▁▁▂▁▂▁
val_acc,▁▂▄▄▅▅▅▅▅▅▅▅▆▇▇▆▆▆▆▇▇▇▇▆▇▇▇██▇▇▇██▇█▇▇▇█
val_loss,█▆▅▅▅▄▄▄▃▄▃▃▃▃▃▂▂▃▂▂▂▂▂▃▁▂▁▂▂▂▂▂▁▂▁▁▂▁▁▁
acc,3.76827
loss,0.11544
val_acc,3.39062
val_loss,0.13519


acc,▁▄▄▄▅▄▆▆▅▆▆▅▇▆▆▆▇▆▆▇▆▆▇▇█▆▆▇▇▇▇▇▇▇█▇▇█▇█
loss,█▅▅▅▄▄▃▃▃▃▃▂▃▃▃▃▃▂▃▂▂▂▂▃▂▂▂▂▂▁▂▂▂▂▁▂▂▂▂▁
val_acc,▁▃▄▄▄▅▅▅▆▅▆▅▇▇▆▇▇▇▇▇▇▇▆▇▇▆▇█▇▇▇▇▇▇▇▇▇█▇▇
val_loss,█▇▅▆▆▄▅▃▄▃▃▄▄▃▃▃▃▃▂▃▃▃▃▃▂▂▃▃▃▁▂▂▂▂▂▂▁▁▁▁
acc,3.59554
loss,-0.10514
val_acc,3.37636
val_loss,-0.01561



# 👀 Important! Click On Link To Your Project And Look Around 👀

Run the below cell to get a link to your project:

In [6]:
from IPython.display import Markdown
Markdown(f"## Your Project's url: https://wandb.ai/{ENTITY}/{PROJECT}")

## Your Project's url: https://wandb.ai/piedpipers/cicd-quickstart"